In [1]:
!nvidia-smi

Wed Feb 24 17:28:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 450.57       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:18:00.0 Off |                  N/A |
| 32%   50C    P0    65W / 250W |      3MiB / 11016MiB |     15%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf
from tensorflow.keras import utils, datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import os
import math
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import wandb

In [3]:
os.getcwd()
os.environ['CUDA_VISIBLE_DEVICES'] = "0" 
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)]) 
    except RuntimeError as e:
        print(e)
wandb.init("cifar-project")


Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.20 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
EPOCH = 200
BATCH_SIZE = 32
IMAGE_SIZE_CROPPED = 24
LR = 1e-4

scce = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.SUM)
optimizer = tf.keras.optimizers.Adam(lr = LR)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [5]:
# load data
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()

# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

print('X_train shape:', X_train.shape)
print('Y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('Y_test shape:', y_test.shape)

X_train shape: (50000, 32, 32, 3)
Y_train shape: (50000, 1)
X_test shape: (10000, 32, 32, 3)
Y_test shape: (10000, 1)


In [6]:
# parse training data
def map_fun(image, label):    distorted_image = tf.image.random_crop(image, [IMAGE_SIZE_CROPPED,IMAGE_SIZE_CROPPED,3])
    distorted_image = tf.image.random_flip_left_right(distorted_image)
    distorted_image = tf.image.random_brightness(distorted_image, max_delta=63)
    distorted_image = tf.image.random_contrast(
        distorted_image, lower=0.2, upper=1.8)
    distorted_image = tf.image.per_image_standardization(distorted_image)
    return distorted_image, label

# parse testing data
def map_fun_test(image, label):
    distorted_image = tf.image.resize_with_crop_or_pad(image, IMAGE_SIZE_CROPPED,IMAGE_SIZE_CROPPED)
    distorted_image = tf.image.per_image_standardization(distorted_image)
    return distorted_image, label

In [7]:
dataset_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset_train = dataset_train.map(map_fun)
dataset_train = dataset_train.shuffle(1000)
dataset_train = dataset_train.batch(BATCH_SIZE)

dataset_test = tf.data.Dataset.from_tensor_slices((X_test, y_test))
dataset_test = dataset_test.map(map_fun_test)
dataset_test = dataset_test.shuffle(1000)
dataset_test = dataset_test.batch(BATCH_SIZE)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [8]:
class CNN_Model(tf.keras.Model):
    def __init__(self):
        super(CNN_Model, self).__init__()
        self.cnn1 = tf.keras.layers.Conv2D(64, (5,5), padding='same', activation='relu', input_shape = (32, 32, 3))
        self.maxpool1 = tf.keras.layers.MaxPool2D(pool_size=(3,3),strides=(2,2))
        self.cnn2 = tf.keras.layers.Conv2D(64, (5,5), padding='same', activation='relu')
        self.maxpool2 = tf.keras.layers.MaxPool2D(pool_size=(3,3),strides=(2,2))

        self.flat = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(384, activation='relu')
        self.drop = tf.keras.layers.Dropout(0.5)
        self.dense2 = tf.keras.layers.Dense(192, activation='relu')
        self.dense3 = tf.keras.layers.Dense(10, activation='softmax')
        self.batchnorm = tf.keras.layers.BatchNormalization()
        
    def call(self, inputs):
        x = self.cnn1(inputs)
        x = self.maxpool1(x)
        x = self.batchnorm(x)
        
        x = self.cnn2(x)
        x = self.maxpool2(x)
        x = self.batchnorm(x)

        x = self.flat(x)
        x = self.dense1(x)
        x = self.drop(x)
        x = self.dense2(x)
        x = self.drop(x)
        x = self.dense3(x)
        return x
cnn_model = CNN_Model()

In [9]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = cnn_model(images)
        loss = scce(labels, predictions)
    grads = tape.gradient(loss, cnn_model.trainable_weights)
    optimizer.apply_gradients(zip(grads, cnn_model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)
@tf.function
def val_step(images, labels):
    preds = cnn_model(images)
    loss = scce(labels, preds)
    test_loss(loss)
    test_accuracy(labels, preds)

In [10]:
# training step 
for ep in range(EPOCH):
    for (train_img, train_label), (val_img, val_label) in zip(dataset_train,dataset_test):
        train_step(train_img, train_label)
        val_step(val_img, val_label)
    wandb.log({"train accuracy": train_accuracy.result().numpy(),
               "train loss " : train_loss.result().numpy(),
               "val accuracy " : test_accuracy.result().numpy(),
               "val loss " : test_loss.result().numpy()})
    if ep % 10 == 0: 
        tmp = "Epoch {:0}, train accuracy {:.2f}, train loss {:.2f}, val accuracy {:.2f}, val loss {:.2f}"
        print(tmp.format(ep,
                         train_accuracy.result().numpy(),
                         train_loss.result().numpy(),
                         test_accuracy.result().numpy(),
                         test_loss.result().numpy()))
train_loss.reset_states()
train_accuracy.reset_states()
test_loss.reset_states()
test_accuracy.reset_states()

Epoch 0, train accuracy 0.28, train loss 62.11, val accuracy 0.31, val loss 60.22
Epoch 10, train accuracy 0.48, train loss 46.67, val accuracy 0.51, val loss 44.02
Epoch 20, train accuracy 0.54, train loss 41.35, val accuracy 0.57, val loss 39.22
Epoch 30, train accuracy 0.58, train loss 37.78, val accuracy 0.60, val loss 36.41
Epoch 40, train accuracy 0.61, train loss 35.00, val accuracy 0.62, val loss 34.49
Epoch 50, train accuracy 0.64, train loss 32.75, val accuracy 0.64, val loss 33.10
Epoch 60, train accuracy 0.66, train loss 30.82, val accuracy 0.65, val loss 32.11
Epoch 70, train accuracy 0.68, train loss 29.16, val accuracy 0.66, val loss 31.37
Epoch 80, train accuracy 0.70, train loss 27.66, val accuracy 0.67, val loss 30.81
Epoch 90, train accuracy 0.71, train loss 26.33, val accuracy 0.68, val loss 30.44
Epoch 100, train accuracy 0.72, train loss 25.12, val accuracy 0.68, val loss 30.18
Epoch 110, train accuracy 0.74, train loss 24.01, val accuracy 0.69, val loss 30.04
Epo